In [ ]:
# JSON Serving API実行（競技システム用）
# このセルは競技システムが実行時に使用
print("=== JSON Serving API起動 ===")
print("STDIN/STDOUT モードで待機中...")

sys.stdout.reconfigure(line_buffering=True)

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue

    try:
        # JSON入力解析
        payload = json.loads(line)
        series_id = payload.get("SeriesInstanceUID") or payload.get("series_id")

        # 予測実行
        predictions = predict_14_labels(gbm, metadata, payload)

        # JSON応答
        response = {
            "SeriesInstanceUID": series_id,
            "predictions": predictions
        }

        # STDOUT出力
        sys.stdout.write(json.dumps(response) + "\n")
        sys.stdout.flush()

    except Exception as e:
        # エラー応答
        error_response = {
            "error": str(e),
            "SeriesInstanceUID": payload.get("SeriesInstanceUID", "unknown")
        }
        sys.stdout.write(json.dumps(error_response) + "\n")
        sys.stdout.flush()

In [ ]:
# モデル読み込みとテスト実行
print("=== モデル読み込み ===")
gbm, metadata = load_model_files()

print(f"GBMモデル: {type(gbm)}")
print(f"メタデータキー: {list(metadata.keys())}")
print(f"mod_columns: {len(metadata.get('mod_columns', []))} items")

# テストデータで動作確認
test_data = {
    "SeriesInstanceUID": "test_001",
    "PatientAge": "60 - 65",
    "PatientSex": "Male",
    "Modality": "CT"
}

print("\n=== テスト予測実行 ===")
predictions = predict_14_labels(gbm, metadata, test_data)

print(f"テストシリーズ: {test_data['SeriesInstanceUID']}")
print("予測結果（上位5ラベル）:")
for i, (label, prob) in enumerate(predictions.items()):
    if i < 5:
        print(f"  {label}: {prob:.4f}")
print("...")
print(f"合計ラベル数: {len(predictions)}")
print(f"Aneurysm Present確率: {predictions['Aneurysm Present']:.4f}")

In [ ]:
# ユーティリティ関数定義（kaggle_utilsの主要機能を統合）

# 14ラベル定義
LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present'
]

def load_model_files():
    """モデルファイルを読み込み"""
    # GBMモデル読み込み
    model_path = os.path.join(MODEL_DIR, "GBM Baseline.pkl")
    with open(model_path, 'rb') as f:
        gbm = pickle.load(f)

    # メタデータ読み込み
    metadata_path = os.path.join(MODEL_DIR, "Complete Workflow Metadata.json")
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)

    return gbm, metadata

def build_features(series_meta, mod_columns, age_median=60.0):
    """特徴量ベクトル構築"""
    # 年齢処理
    age_str = str(series_meta.get('PatientAge', ''))
    try:
        age_val = float(age_str.split(' - ')[0])
    except:
        age_val = age_median

    # 性別処理
    sex_val = 1.0 if series_meta.get('PatientSex', '') == 'Male' else 0.0

    # モダリティone-hot
    modality = series_meta.get('Modality', '')
    mod_feats = []
    for mod_col in mod_columns:
        mod_feats.append(1.0 if mod_col == f"mod_{modality}" else 0.0)

    # 特徴量配列
    feats = np.array([age_val, sex_val] + mod_feats, dtype=np.float32)
    return feats.reshape(1, -1)

def predict_14_labels(gbm, metadata, series_meta):
    """14ラベル予測実行"""
    mod_columns = metadata.get('mod_columns', [])
    age_median = float(metadata.get('age_median', 60.0))

    # 特徴量構築
    feats = build_features(series_meta, mod_columns, age_median)

    # GBM予測（Aneurysm Presentの確率）
    presence_prob = float(gbm.predict_proba(feats)[:, 1][0])

    # 全14ラベルに同じ確率を適用（ベースライン実装）
    predictions = {label: presence_prob for label in LABEL_COLS}

    return predictions

print("=== ユーティリティ関数定義完了 ===")

# RSNA-2025 実提出ノートブック (exp0001_baseline)

このノートブックは学習済みGBMモデル（年齢・性別・モダリティ）から14ラベル予測を実行し、Kaggle競技に提出します。

## 実行手順
1. 環境セットアップとライブラリ読み込み
2. モデル読み込みとテスト実行
3. JSON Serving API 起動（競技システム用）

## 使用データセット
- exp001-rsna: 学習済みGBMモデルとメタデータ

In [ ]:
# JSON Serving API実行（競技システム用）
# このセルは競技システムが実行時に使用
print("=== JSON Serving API起動 ===")
print("STDIN/STDOUT モードで待機中...")

# Kaggle環境ではsys.stdout.reconfigure()が使えないため削除
# sys.stdout.reconfigure(line_buffering=True)

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue

    try:
        # JSON入力解析
        payload = json.loads(line)
        series_id = payload.get("SeriesInstanceUID") or payload.get("series_id")

        # 予測実行
        predictions = predict_14_labels(gbm, metadata, payload)

        # JSON応答
        response = {
            "SeriesInstanceUID": series_id,
            "predictions": predictions
        }

        # STDOUT出力（flushで即座に出力）
        print(json.dumps(response))
        sys.stdout.flush()

    except Exception as e:
        # エラー応答
        error_response = {
            "error": str(e),
            "SeriesInstanceUID": payload.get("SeriesInstanceUID", "unknown")
        }
        print(json.dumps(error_response))
        sys.stdout.flush()